In [1]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import xarray as xr
# import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import os

# from Testing_Stacks import *

In [13]:
lat_bnds, lon_bnds = [6.5, 38.5], [66.5, 100]
projection_dimension = [129, 135]
lat_lst = np.linspace(lat_bnds[0], lat_bnds[1], projection_dimension[0])
lon_lst = np.linspace(lon_bnds[0], lon_bnds[1], projection_dimension[1])

In [14]:
CanESM2 = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/CanESM2.nc")
HadGEM2_ES = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/HadGEM2-ES.nc")
MIROC_ESM = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/MIROC-ESM.nc")
GFDL_CM3 = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/GFDL-CM3.nc")
dates_CanESM2 = CanESM2.coords['time'].to_numpy()[4:]
dates_HadGEM2_ES = HadGEM2_ES.coords['time'].to_numpy()[4:]
dates_MIROC_ESM = MIROC_ESM.coords['time'].to_numpy()[4:]
dates_GFDL_CM3 = GFDL_CM3.coords['time'].to_numpy()[4:]
print(len(dates_CanESM2))
print(len(dates_HadGEM2_ES))
print(len(dates_MIROC_ESM))
print(len(dates_GFDL_CM3))

21166
20730
21166
21166


In [15]:
for gcm, dates, y in zip(['MIROC-ESM', 'HadGEM2-ES', 'GFDL-CM3', 'CanESM2'],[dates_MIROC_ESM, dates_HadGEM2_ES, dates_GFDL_CM3, dates_CanESM2], ['Y', 'Y_360', 'Y', 'Y']):
    pr_values = np.load(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/04_Testing_Stacks/npy_Files/yy_hat_{gcm}.npy").reshape(-1,129,135)
    Y = np.load(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/Stacked/{y}.npy").reshape(-1,129,135)[4:]
    pr_values = (pr_values*Y.std())+Y.mean()
    ds = xr.Dataset({
            'pr': xr.DataArray(
                data = pr_values.reshape(len(dates), len(lat_lst), len(lon_lst)),
                dims = ['time', 'lat', 'lon'],
                coords = {
                    'time': dates,
                    'lat': lat_lst,
                    'lon': lon_lst
                },
                attrs = {'units':'mm/day'}
            )
        }
    )
    ds.to_netcdf(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/04_Testing_Stacks/NC_Files/Actual_{gcm}.nc")

In [4]:
lat_bnds, lon_bnds = [25, 50], [-125, -70]
lat_lst = np.arange(lat_bnds[0], lat_bnds[1]+0.01, 0.5)
lon_lst = np.arange(lon_bnds[0], lon_bnds[1]+0.01, 0.5)
projection_dimension = [129, 135]

In [17]:
CanESM2 = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/CanESM2_mon.nc")
HadGEM2_ES = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/HadGEM2-ES_mon.nc")
MIROC_ESM = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/MIROC-ESM_mon.nc")
GFDL_CM3 = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/GFDL-CM3_mon.nc")
dates_CanESM2 = CanESM2.coords['time'].to_numpy()[4:]
dates_HadGEM2_ES = HadGEM2_ES.coords['time'].to_numpy()[4:]
dates_MIROC_ESM = MIROC_ESM.coords['time'].to_numpy()[4:]
dates_GFDL_CM3 = GFDL_CM3.coords['time'].to_numpy()[4:]
print(len(dates_CanESM2))
print(len(dates_HadGEM2_ES))
print(len(dates_MIROC_ESM))
print(len(dates_GFDL_CM3))

692
691
692
692


In [19]:
for gcm, dates, y in zip(['MIROC-ESM', 'HadGEM2-ES', 'GFDL-CM3', 'CanESM2'],[dates_MIROC_ESM, dates_HadGEM2_ES, dates_GFDL_CM3, dates_CanESM2], ['Y', 'Y', 'Y', 'Y']):
    pr_values = np.load(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/04_Testing_Stacks/npy_Files/yy_hat_{gcm}_mon.npy").reshape(-1,129,135)
    Y = np.load(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/Stacked/{y}_mon.npy").reshape(-1,129,135)[4:]
    pr_values = (pr_values*Y.std())+Y.mean()
    if gcm == 'HadGEM2-ES':
        pr_values = np.array(pr_values[:-1,39:90,12:123])
    else:
        pr_values = np.array(pr_values[:,39:90,12:123])
    ds = xr.Dataset({
            'pr': xr.DataArray(
                data = pr_values.reshape(len(dates), len(lat_lst), len(lon_lst)),
                dims = ['time', 'lat', 'lon'],
                coords = {
                    'time': dates,
                    'lat': lat_lst,
                    'lon': [x%360 for x in lon_lst]
                },
                attrs = {'units':'mm/month'}
            )
        }
    )
    ds.to_netcdf(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/04_Testing_Stacks/NC_Files/Actual_{gcm}_mon.nc")

In [5]:
CanESM2 = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/CanESM2_mon_temp.nc")
HadGEM2_ES = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/HadGEM2-ES_mon_temp.nc")
MIROC_ESM = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/MIROC-ESM_mon_temp.nc")
GFDL_CM3 = xr.open_dataset("/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/NC_Files/GFDL-CM3_mon_temp.nc")
dates_CanESM2 = CanESM2.coords['time'].to_numpy()[4:]
dates_HadGEM2_ES = HadGEM2_ES.coords['time'].to_numpy()[4:]
dates_MIROC_ESM = MIROC_ESM.coords['time'].to_numpy()[4:]
dates_GFDL_CM3 = GFDL_CM3.coords['time'].to_numpy()[4:]
print(len(dates_CanESM2))
print(len(dates_HadGEM2_ES))
print(len(dates_MIROC_ESM))
print(len(dates_GFDL_CM3))

692
691
692
692


In [6]:
for gcm, dates, y in zip(['MIROC-ESM', 'HadGEM2-ES', 'GFDL-CM3', 'CanESM2'],[dates_MIROC_ESM, dates_HadGEM2_ES, dates_GFDL_CM3, dates_CanESM2], ['Y', 'Y', 'Y', 'Y']):
    pr_values = np.load(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/04_Testing_Stacks/npy_Files/yy_hat_{gcm}_mon_temp.npy").reshape(-1,129,135)
    Y = np.load(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/03_Preprocess_Data/Stacked/{y}_mon_temp.npy").reshape(-1,129,135)[4:]
    pr_values = (pr_values*Y.std())+Y.mean()
    if gcm == 'HadGEM2-ES':
        pr_values = np.array(pr_values[:-1,39:90,12:123])
    else:
        pr_values = np.array(pr_values[:,39:90,12:123])
    ds = xr.Dataset({
            'pr': xr.DataArray(
                data = pr_values.reshape(len(dates), len(lat_lst), len(lon_lst)),
                dims = ['time', 'lat', 'lon'],
                coords = {
                    'time': dates,
                    'lat': lat_lst,
                    'lon': [x%360 for x in lon_lst]
                },
                attrs = {'units':'degrees Celsius'}
            )
        }
    )
    ds.to_netcdf(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/04_Testing_Stacks/NC_Files/Actual_{gcm}_mon_temp.nc")